`moth` is a NumPy based implementation of moth-inspired navigation strategies that uses 
`pompy` library to create the puff, wind and concentration models. see `pompy/Readme.md` 
for details

### What is this repository for?

This Python package allows simulation moth-like navigators in dynamic 2D odour 
concentration fields spread in turbulent flows 

### Installation and requirements

    Python 2.7
    Numpy
    Scipy
    Matplotlib

### Example usage
![Demo flight](moth/Demonstration_of_different_navigation_strategies.png)


The following code simulates a meandering plume, runs naviagatores through the plume simulation, and prints out the navigation statistics of the run. 
Firstly, the function generate_job generates the job.json file that holds all of the needed information to run a simulation, divided to sections. 

In [6]:
from __future__ import division
import json
def generate_job(char_time =3.5,amplitude =0.1,
                 job_file='job.json', puff_release_rate = 10, puff_spread_rate =0.003,
                  t_max =75., dt = 0.01,num_it =1, base_duration =0.2):
    const_dict={}

    #simulation
    const_dict['num_it'] = num_it
    const_dict['t_max'] = t_max
    const_dict['dt'] = dt

    #wind meandering
    const_dict['char_time'] = char_time
    const_dict['amplitude'] = amplitude

    #plume source
    const_dict['puff_release_rate'] = puff_release_rate
    const_dict['puff_spread_rate'] = puff_spread_rate
    
    #navigators
    const_dict['x_start'] = 100
    const_dict['y_start'] = 500
    const_dict['nav_type'] = 'alex'
    const_dict['wait_type'] = 1
    const_dict['base_turn_angle'] = 18 #for crw
    const_dict['threshold'] = 500
    const_dict['duration'] = base_duration
    
    with open(job_file, 'w') as outfile:
        json.dump(const_dict, outfile)

In [ ]:
this file creates trajectory data
note - In this file we only define the relvant function, there is no actual output

In [8]:
from __future__ import division

__authors__ = 'Noam Benelli'
import models
from simulation import moth_simulation
import copy


The following function is the heart and soul of this file -
the waiting, casting and navigating strategies can vary between different navigator groups, but the 

#### Set up the navigators (optional) 
The file Casting_competition initiates the navigators to run in the simulation. The function call_navigators sets two hundred navigators of a certein waiting, casting and navigating type. The only difference between the navigators is their initial location - navigator.x, navigator.y

In [18]:
def call_navigators(navigator1,wait,cast,nav):
        for j in range(10):
            for i in range(20):
                global navigators
                new_navigator = copy.copy(navigator1)
                new_navigator.wait_type = wait
                new_navigator.cast_type = cast
                new_navigator.nav_type = nav
                new_navigator.y = 450 - i*3
                new_navigator.x = 499 - j

            title =  \
                ' cast - ' + str(new_navigator.cast_type) \
                + '; nav - ' + str(new_navigator.nav_type) \
                + str(len(navigators))
                
            navigators.append(new_navigator)
            navigator_titles.append(title)

The next segment opens the job.json file and initiates a navigator object (moth_modular) using the predefined parameters. 
Afterwards it calls the function defined in the last segment (call_navigators), given four sets of parameters.

In [19]:
def create_trajectory_data(job_file_name = 'job.json',data_file_name ='data.json',
                           titles_file_name = 'titles.json'):
    with open(job_file_name) as data_file:
        cd = json.load(data_file) #constants dictionary
    sim_region = models.Rectangle(0., -1., 4., 1.)


    #call the base navigator, using the parameters defined in the job file 
    #the competing navigators in the simulation retain
    #any parameter of navigator1 that isn't changed in call_navigators
    navigator1 = models.moth_modular(sim_region, cd['x_start'], cd['y_start'],cd['nav_type'] , 2, cd['wait_type'])
    navigator1.base_turn_angle = cd['base_turn_angle']
    navigator1.threshold = cd['threshold']
    navigator1.duration = cd['duration']

    
    #set up a large number of navigators with different properties
    navigators = []
    navigator_titles = []
    #the parameters of the next function calls will be the parameters of the simulated navigators
    #note that the order in which they are called will the order in which they will be presented later on
    call_navigators(navigator1,1,2,'alex')
    call_navigators(navigator1,1,3,'alex')
    call_navigators(navigator1,1,'carde2',1)
    call_navigators(navigator1,1,'carde1',1)
     
    #run the simulation - each navigator runs through the exact same plume conditions
    dict_list = moth_simulation(cd['num_it'],
                                navigators,cd['t_max'],cd['char_time'],
                                cd['amplitude'], cd['dt'], cd['puff_release_rate'],
                                cd['puff_spread_rate'],
                                1,
                                False)


    with open(data_file_name, 'w') as outfile:
        json.dump(dict_list, outfile)
    return navigator_titles



### 

In [4]:
from graphics_from_file import save_plot, save_detection_plot


#### Set up the wind and plume conditions (optional) 
The next segment initiates the main loop. For each iteration a new plume and wind model are initiated for the navigators to be simulated in. The function generate_job dictates the terms of the simulation in terms of wind and plume partameteres. In order to set the simulation enter the required parameters as input for generate_job. Those will be written into the corrsponding job files. 
In order to create a gradient between the iterations, define the parameter using the loop index i.
For example, as the simulation set now, the only value that changes is the puff spread rate, varying from 0.0001 to 0.0004.
Make sure that only one variable of the simulation changes with each iteration.
#### Run The simulations
When the file is run the wind and plume paramters that have been set are saved into "job" files, one JSON file for each iteration (job0.JSON, job1.JSON...). 
The trajectories of the navigators are saved as "data" files, (data0.JSON, data1.JSON), on which the later analyses will be made. 
Notice the following line - 
```
#save_plot(job_file_name,data_file_name,title,navigator_titles)
```
deleting the # mark would instruct the code to save an image for each navigation attempt in the deafulat settings, that mean 800 pictures). That could supply useful input in some cases. 
   

In [20]:
if __name__ == "__main__":
    for i in range(4):
        job_file_name = 'job'+ str(i)+ '.json'
        data_file_name = 'data'+ str(i)+ '.json'
        generate_job(char_time = 7, amplitude = 0.1, job_file = job_file_name,
                     t_max =20, puff_release_rate = 100,
                     puff_spread_rate = 0.0001*(1+i),
                     dt = 0.01, num_it = 1)
        navigator_titles = create_trajectory_data(job_file_name,data_file_name)
        title = 'loop ' +str(i) 
        #save_plot(job_file_name,data_file_name,title,navigator_titles)
        #save_detection_plot(job_file_name,data_file_name,navigators_titles)
        print ('finished simulation number ' + str(i+1))
        

NameError: global name 'navigators' is not defined

In [24]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from bars import get_data

__authors__ = 'Noam Benelli'

matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15)


def create_line_graphs(tot_stats,int_loop):
    # nav_titles = ('Liberzon','Benelli','Large Final Sweeps','Final Sweeps')
    # colors = ('green','red','blue','yellow')
    # loop = str(int_loop)

    plt.figure()
    for stats in tot_stats:
        plt.plot(stats)
        # plt.ylabel('')

def detect_change(job_list):
    for key in job_list[0]:
        if job_list[0][key] != job_list[1][key]:
            sig_key = key
            break
    value_list =[]
    for job in job_list:
        value_list.append(job[sig_key])
        
    return (sig_key, value_list)

def process_jobs(num_jobs):
    job_list = []
    for i in range(num_jobs):
        file_name = 'job'+str(i)+'.json'
        with open(file_name) as data_file1:  
            job = json.load(data_file1)
        job_list.append(job)
    return detect_change(job_list)
def present_graphs():
    num_jobs = 4
    (xlabel,values)=process_jobs(num_jobs)
    legends = ('A','B','C','D')
    liberzonlist =[]
    Benellilist = []
    lfslist =[]
    fslist = []
    for i in range(num_jobs):
        loop = str(i)
        data_list = get_data('data'+loop+'.json',4)
        #print (data_list[0])
        liberzonlist.append(data_list[0])
        Benellilist.append((data_list[1]))
        lfslist.append((data_list[2]))
        fslist.append((data_list[3]))
    #[succ_prec ,average_time_,average_efficiency]
    lib_succ, lib_avg,lib_efficiency = zip(*liberzonlist)
    Bene_succ, Bene_avg,Bene_efficiency = zip(*Benellilist)
    lfs_succ, lfs_avg, lfs_efficiency = zip(*lfslist)
    fs_succ, fs_avg,fs_efficiency = zip(*fslist)


    fig,ax = plt.subplots()

    n_groups = 4
    bar_width = 0.3
    opacity = 0.4
    index = np.arange(0, 2*n_groups, 2)
    """
    data0 =(lib_succ[0],Bene_succ[0],lfs_succ[0],fs_succ[0])
    data1 =(lib_succ[1],Bene_succ[1],lfs_succ[1],fs_succ[1])
    data2 =(lib_succ[2],Bene_succ[2],lfs_succ[2],fs_succ[2])
    data3 =(lib_succ[3],Bene_succ[3],lfs_succ[3],fs_succ[3])
    """
    def set_charts(data0,data1,data2,data3):
        global chart
        chart = plt.bar(index, data0, bar_width,color = 'white', edgecolor='black')
        chart = plt.bar(index+bar_width, data1, bar_width,color = 'white', hatch = '+' ,edgecolor='black')
        chart = plt.bar(index+2*bar_width, data2, bar_width,color = 'white', hatch = '\\', edgecolor='black')
        chart = plt.bar(index+3*bar_width, data3, bar_width,color = 'black', hatch = '*', edgecolor='black')
    set_charts(lib_succ,Bene_succ,lfs_succ,fs_succ)
    if xlabel == 'puff_spread_rate':
        ax.set_xlabel(r'$\sigma$')
    else:
        ax.set_xlabel(xlabel)
    ax.set_ylabel('Success (%)')
    ax.xaxis.label.set_fontsize(17)
    ax.yaxis.label.set_fontsize(17)
    #ax.set_title('Success Percentage vs '+ xlabel)
    plt.xticks(index+bar_width*1.5, (str(values[0]), str(values[1]), str(values[2]),str(values[3])))
    plt.legend(legends)
    plt.tight_layout()


    plt.savefig('Success Percentage vs '+ xlabel)
    plt.show()
    
    #create second graph
    fig,ax = plt.subplots()

    n_groups = 4
    bar_width = 0.3
    opacity = 0.4
    index = np.arange(0, 2*n_groups, 2)
    
    #plt.xlabel(xlabel)
    #plt.ylabel('Average Navigation Time (ratio)')
    if xlabel == 'puff_spread_rate':
        ax.set_xlabel(r'$\sigma$')
    else:
        ax.set_xlabel(xlabel)
    ax.set_ylabel('T/'+r'$\tau$')

    
    set_charts(lib_avg,Bene_avg,lfs_avg,fs_avg)
    plt.subplots_adjust(bottom=0.15)
    plt.xticks(index+bar_width*1.5, (str(values[0]), str(values[1]), str(values[2]),str(values[3])))
    #ax.set_title('Average Navigation Time vs '+ xlabel)
    ax.xaxis.label.set_fontsize(17)
    ax.yaxis.label.set_fontsize(17)
    plt.legend(legends)
    plt.savefig('Average Navigation Time vs '+ xlabel)
    plt.show()
present_graphs()